In [1]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from surprise import Dataset, Reader, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise import dump
from sklearn.preprocessing import normalize

In [2]:
movies_df = pd.read_csv("D:\\Year 3 Semester 2\\computerScienceProject\\tmdb_5000_movies.csv")
credits_df = pd.read_csv("D:\\Year 3 Semester 2\\computerScienceProject\\tmdb_5000_credits.csv")
ratings_df = pd.read_excel("D:\\Year 3 Semester 2\\computerScienceProject\\movie_ratings.xlsx")

In [3]:
#merge the two datasets
movies_df = movies_df.merge(credits_df, on='title')
movies_df = movies_df.merge(ratings_df, on='movie_id')

In [4]:
#selecting relevant columns
movies_final = movies_df[[ 'id', 'title', 'genres', 'runtime', 'overview', 'keywords', 'cast', 'crew', 'User_ID', 'RATINGS']]

In [5]:
#check for null values
print(movies_final.isna().sum())

id          0
title       0
genres      0
runtime     2
overview    3
keywords    0
cast        0
crew        0
User_ID     0
RATINGS     1
dtype: int64


In [6]:
#drop null values
movies_final.dropna(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\326592079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final.dropna(inplace=True)


In [7]:
#function to extract genre name
def convert(obj):
    l = []

    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

movies_final['genres'] = movies_final['genres'].apply(convert)
movies_final['keywords'] = movies_final['keywords'].apply(convert)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\4245063021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['genres'] = movies_final['genres'].apply(convert)
C:\Users\HP\AppData\Local\Temp\ipykernel_13748\4245063021.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['keywords'] = movies_final['keywords'].apply(convert)


In [8]:
#function to extract first three cast names
def convert3(obj):
    l = []
    counter = 0

    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

movies_final['cast'] = movies_final['cast'].apply(convert3)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\1541414322.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['cast'] = movies_final['cast'].apply(convert3)


In [9]:
# to extract director's name
def extract_director(obj):
    l = []

    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

movies_final['crew'] = movies_final['crew'].apply(extract_director)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\550475311.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['crew'] = movies_final['crew'].apply(extract_director)


In [10]:
movies_final['overview'] = movies_final['overview'].apply(lambda x: x.split()) 

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\2814892025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['overview'] = movies_final['overview'].apply(lambda x: x.split())


In [11]:
# removing spaces between words
movies_final['genres'] = movies_final['genres'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['keywords'] = movies_final['keywords'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['cast'] = movies_final['cast'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['crew'] = movies_final['crew'].apply(lambda x : [i.replace(" ", "") for i in x])

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\361890825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['genres'] = movies_final['genres'].apply(lambda x : [i.replace(" ", "") for i in x])
C:\Users\HP\AppData\Local\Temp\ipykernel_13748\361890825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['keywords'] = movies_final['keywords'].apply(lambda x : [i.replace(" ", "") for i in x])
C:\Users\HP\AppData\Local\Temp\ipykernel_13748\361890825.py:4: SettingWithCopyWarning: 
A valu

In [12]:
movies_final['tags'] = movies_final['genres'] + movies_final['keywords'] + movies_final['cast'] + movies_final['crew'] + movies_final['overview']

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\451815744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['tags'] = movies_final['genres'] + movies_final['keywords'] + movies_final['cast'] + movies_final['crew'] + movies_final['overview']


In [13]:
movies = movies_final[['id', 'title', 'tags', 'User_ID', 'RATINGS']]

In [14]:
movies['tags'] = movies['tags'].apply(lambda x : " ".join(x))
movies['tags'] = movies['tags'].apply(lambda x : x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\3752147239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x : " ".join(x))
C:\Users\HP\AppData\Local\Temp\ipykernel_13748\3752147239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x : x.lower())


In [15]:
movies['RATINGS'] = movies['RATINGS'].replace(0, 1)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\1226471429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['RATINGS'] = movies['RATINGS'].replace(0, 1)


In [16]:
reader = Reader(rating_scale = (1, 10))
data = Dataset.load_from_df(movies[['id', 'User_ID', 'RATINGS']], reader)
traindf = data.build_full_trainset()
testdf = traindf.build_anti_testset()

In [17]:
sim_options = {'name' : 'cosine',
               'user_based' : False  #defining similarity measures
              }
               
knnbaseline_model = KNNBaseline(sim_options=sim_options)

knnbaseline_model.fit(traindf)
knnbaseline_predictions = knnbaseline_model.test(testdf)
accuracy.rmse(knnbaseline_predictions)
accuracy.mae(knnbaseline_predictions)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.3985
MAE:  0.3126


0.3125622128537393

In [18]:
svdpp_model = SVDpp()

svdpp_model.fit(traindf)
svdpp_model_predictions = svdpp_model.test(testdf)

accuracy.rmse(svdpp_model_predictions)
accuracy.mae(svdpp_model_predictions)

MemoryError: 

In [ ]:
#  content-based filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
content_matrix = tfidf_vectorizer.fit_transform(movies['tags'])
content_similarity = linear_kernel(content_matrix, content_matrix)

In [ ]:
mappings = pd.Series(movies.index, index=movies['title'])

In [ ]:
def get_recommendations_new(title):
    idx = mappings[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movies['id'].iloc[movie_indices]

In [ ]:
# def hybrid(userId):
#     user_movies = testdf[testdf['User_ID'] == userId]
#     user_movies['est'] = user_movies['id'].apply(lambda x: 0.6*knnbaseline_algo.predict(userId,x).est + 0.4*svdpp_algo.predict(userId, x).est)    
#     user_movies = user_movies.sort_values(by ='est', ascending=False).head(4)
#     user_movies['Model'] = 'SVD + CF'
# #     user_movies = user_movies['movieId'].values.tolist()
# #     print("User liked movies list: ", user_movies)
    
#     recommend_list = user_movies[['movieId', 'est', 'Model']]
#     print(recommend_list.head())

# #     top_movie = user_movies['movieId'].iloc[0]
# #     print("Top movie id", top_movie)
# #     top_movie_title = movies['title'][movies['movieId'] == top_movie].values[0]
# #     print("Top movie title", top_movie_title)

    
#     movie_list = recommend_list['movieId'].values.tolist()
#     print(movie_list)
#     sim_movies_list = []
#     for movie_id in movie_list:
#         # Call content based 
#         movie_title = movies['title'][movies['movieId'] == movie_id].values[0]
#         sim_movies = get_recommendations_new(movie_title)
# #         print(sim_movies.values.tolist())
#         sim_movies_list.extend(sim_movies)
    
    
#     # Compute ratings for the popular movies
#     for movie_id in sim_movies_list:
#         pred_rating = 0.6*knnbaseline_algo.predict(userId, movie_id).est + 0.4*svdpp_algo.predict(userId, movie_id).est
#         row_df = pd.DataFrame([[movie_id, pred_rating, 'Movie similarity']], columns=['movieId', 'est','Model'])
#         recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
    
#     # Popular based movies
#     top_genre_list = user_top_genre(userId)
#     print("User top genre list: ", top_genre_list)
    
#     popular_movies = []
#     for top_genre in top_genre_list:
#         popular_movies.extend(genre_based_popularity(top_genre))
#     print("Final list: ", popular_movies)
    
#     # Compute ratings for the popular movies
#     for movie_id in popular_movies:
#         pred_rating = 0.6*knnbaseline_algo.predict(userId, movie_id).est + 0.4*svdpp_algo.predict(userId, movie_id).est
#         row_df = pd.DataFrame([[movie_id, pred_rating, 'Popularity']], columns=['movieId', 'est','Model'])
#         recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
#     recommend_list = recommend_list.drop_duplicates(subset=['movieId'])
#     train_movie_list = traindf[traindf['userId']==userId]['movieId'].values.tolist()
    
#     # Remove movies in training for this user
#     mask = recommend_list.movieId.apply(lambda x: x not in train_movie_list)
#     recommend_list = recommend_list[mask]
    
#     return recommend_list